In [1]:
# Python 2
from __future__ import print_function
import numpy as np
np.set_printoptions(threshold=np.nan, linewidth=np.nan)

In [2]:
import pickle
with open('df.pickle', mode='rb') as h:
    df = pickle.load(h)
with open('hex.pickle', mode='rb') as h:
    hex_data = pickle.load(h)
id_name = hex_data['id_name']

In [3]:
import cv2  # loading images with OpenCV is much faster than with Caffe
import h5py
from os.path import join
img_dir = '../pascal12/JPEGImages'
for split in ['train', 'test']:
    filenames = df[df['split']==split]['img'].reset_index(drop=True)
    file_count = len(filenames)
    X = list()
    for i in range(0, file_count):  # load images
        img = cv2.resize(cv2.imread(join(img_dir, filenames[i])), (256, 256))
        X.append(img.astype(np.float32) / 256)
        progress = int(float(i) / file_count * 1000) / 10.0
        print('{} {}: {}%'.format(split, filenames[i], progress), end='\r')
    X = np.array(X, dtype=np.float32)
    with open('../{}.npy'.format(split), mode='wb') as h:  # mean pixle
        np.save(h, X.mean(axis=(0, 1, 2)))
    Y = np.zeros((len(X), len(id_name)), dtype=np.float16)  # ground truth
    pseudo_labels = df[df['split']==split]['pseudo_label'].reset_index(drop=True)
    for i in range(0, len(Y)):
        Y[i, pseudo_labels[i]] = 1
    with h5py.File('../{}.h5'.format(split), 'w') as h:  # write h5 file
        print('writing {}.h5'.format(split), end='\r')
        h['X'] = X
        for i in range(0, len(id_name)):
            h[id_name[i]] = Y[:, i]
    with open('../{}.txt'.format(split), 'w') as h:  # caffe instructions
        h.write('{}.h5\n'.format(split))